In [19]:
!pip install pandas scikit-learn tensorflow keras imbalanced-learn

You should consider upgrading via the '/home/alexa/codingTime/share_prices_prediction/venv/bin/python -m pip install --upgrade pip' command.


In [38]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

In [21]:
df = pd.read_csv("data/amazon.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1444 entries, 0 to 1443
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          1444 non-null   int64  
 1   Дата                1067 non-null   object 
 2   Время               1067 non-null   object 
 3   Цена до             1067 non-null   float64
 4   Цена после          1067 non-null   float64
 5   Разница в долларах  1067 non-null   float64
 6   Дельта в процентах  1067 non-null   float64
 7   Текст новости       1067 non-null   object 
dtypes: float64(4), int64(1), object(3)
memory usage: 90.4+ KB


In [22]:
for root, _, files in os.walk("data"):
    for filename in files:
        temp_df = pd.read_csv(os.path.join(root, filename))
        df = pd.concat([df, temp_df], axis=0, sort=False)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18447 entries, 0 to 668
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          18447 non-null  int64  
 1   Дата                10873 non-null  object 
 2   Время               10873 non-null  object 
 3   Цена до             10873 non-null  float64
 4   Цена после          10873 non-null  float64
 5   Разница в долларах  10873 non-null  float64
 6   Дельта в процентах  10873 non-null  float64
 7   Текст новости       10873 non-null  object 
dtypes: float64(4), int64(1), object(3)
memory usage: 1.3+ MB


In [23]:
df = df.drop_duplicates(subset=["Текст новости"], keep=False)
df.drop(columns=[df.columns[0], "Разница в долларах", "Дельта в процентах"], axis=1, inplace=True)
df.rename(columns={"Цена до": "price_before", "Цена после": "price_after", "Дата": "date", "Время": "Time", "Текст новости": "news_text"}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3593 entries, 2 to 443
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          3593 non-null   object 
 1   Time          3593 non-null   object 
 2   price_before  3593 non-null   float64
 3   price_after   3593 non-null   float64
 4   news_text     3593 non-null   object 
dtypes: float64(2), object(3)
memory usage: 168.4+ KB


In [24]:
df["absolute_price_difference"] = df["price_after"] - df["price_before"]
df["percentage_price_difference"] = df["absolute_price_difference"] / df["price_before"] * 100
df["price_change_direction"] = np.where(df["absolute_price_difference"] > 0, 1, -1)
df.head()

,date,Time,price_before,price_after,news_text,absolute_price_difference,percentage_price_difference,price_change_direction
2,2022-10-21,11:57:55,144.250107,145.074997,When Can Elon Musk Sell Tesla Shares? Could Mo...,0.824890,0.571847,1
3,2022-10-21,10:43:20,144.574997,144.250107,Google's New Tweaks To Messages May Annoy iPho...,-0.324890,-0.224721,-1
5,2022-10-20,12:06:39,145.115005,144.869995,"Microsoft Takes On Apple And Google, Plans Xbo...",-0.245010,-0.168839,-1
7,2022-10-18,14:45:49,145.339996,142.779999,T-Mobile US' Pricing Power Of 5G Is A Pivot Po...,-2.559997,-1.761385,-1
8,2022-10-18,14:24:43,144.960007,145.339996,Company That Assembles The iPhone Hopes To Eve...,0.379989,0.262134,1


In [25]:
df[df["price_change_direction"] == 1].count()

date                           1793
Time                           1793
price_before                   1793
price_after                    1793
news_text                      1793
absolute_price_difference      1793
percentage_price_difference    1793
price_change_direction         1793
dtype: int64

In [26]:
df[df["price_change_direction"] == -1].count()

date                           1800
Time                           1800
price_before                   1800
price_after                    1800
news_text                      1800
absolute_price_difference      1800
percentage_price_difference    1800
price_change_direction         1800
dtype: int64

In [37]:
news_df = pd.concat([df["news_text"], df["price_change_direction"]], axis=1)
news_df.head()

,news_text,price_change_direction
2,When Can Elon Musk Sell Tesla Shares? Could Mo...,1
3,Google's New Tweaks To Messages May Annoy iPho...,-1
5,"Microsoft Takes On Apple And Google, Plans Xbo...",-1
7,T-Mobile US' Pricing Power Of 5G Is A Pivot Po...,-1
8,Company That Assembles The iPhone Hopes To Eve...,1


In [42]:
X = news_df["news_text"]
y = news_df["price_change_direction"]

In [43]:
#  Split data to train and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=0.2, shuffle = True, random_state = 8)

ValueError: invalid literal for int() with base 10: ' Earnings\n'